In [1]:
import numpy as np
import json
import tensorflow as tf
gpu = tf.config.experimental.list_physical_devices('GPU')[0]
tf.config.experimental.set_memory_growth(gpu, True)

### Helper Function

In [2]:
def load_json(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
    return data

def json_save(filename, data):
    with open(filename, 'w') as f:
        json.dump(data, f, indent= 4)

In [3]:
def encode_input(input_text, input_encoder, sequence_length = 13):
    x = np.zeros((len(input_text), sequence_length, len(input_encoder)), dtype="float32")
    
    for i, inp in enumerate(input_text):
        for t, char in enumerate(inp):
            x[i, t, input_encoder[char]] = 1.0
        x[i, t+1:, input_encoder[' ']] = 1.0
    return x

In [4]:
def encode_target(target_text, target_encoder, sequence_length = 59):
    x = np.zeros((len(target_text), sequence_length, len(target_encoder)), dtype="float32")
    y = np.zeros((len(target_text), sequence_length, len(target_encoder)), dtype="float32")
    
    for i, inp in enumerate(target_text):
        for t, char in enumerate(inp):
            x[i, t, target_encoder[char]] = 1.0
            if t > 0:
                y[i, t - 1, target_encoder[char]] = 1.0
        x[i, t + 1 :, target_encoder[" "]] = 1.0
        y[i, t:, target_encoder[" "]] = 1.0
    return x, y

In [5]:
input_encoder = load_json('data/input-encoder.json')
target_encoder = load_json('data/target-encoder.json')

In [6]:
input_decoder = dict((i, char) for char, i in input_encoder.items())
target_decoder = dict((i, char) for char, i in target_encoder.items())

In [7]:
json_save('data/input_decoder.json', input_decoder)
json_save('data/target_decoder.json', target_decoder)

### *Loading Model*

In [8]:
model = tf.keras.models.load_model('Eng2Fre.h5')

### *Inference Model*

In [9]:
encoder_inputs = model.input[0]  # encoder input
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # encoder layer
encoder_states = [state_h_enc, state_c_enc]
encoder_model = tf.keras.Model(encoder_inputs, encoder_states) # encoder model

In [10]:
decoder_inputs = model.input[1]  # decoder input

In [11]:
decoder_state_input_h = tf.keras.Input(shape=(256,), name="input_3")
decoder_state_input_c = tf.keras.Input(shape=(256,), name="input_4")

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [12]:
decoder_lstm = model.layers[3] # decoder lstm layer

In [13]:
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]

In [14]:
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)

In [15]:
decoder_model = tf.keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

### *Prediction*

In [16]:
def predict(ip, target_decoder, max_tarlength= 123):
    states_value = encoder_model.predict(ip)

    target_seq = np.zeros((1, 1, len(target_decoder)))
    target_seq[0, 0, target_encoder["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""

    while not stop_condition:
        output, h, c = decoder_model.predict([target_seq] + states_value)

        output_index = np.argmax(output[0, -1, :])
        pred_char = target_decoder[output_index]
        decoded_sentence += pred_char

        if pred_char == "\n" or len(decoded_sentence) > max_tarlength:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, len(target_decoder)))
        target_seq[0, 0, output_index] = 1.0
        # Update states
        states_value = [h, c]
    return decoded_sentence

In [18]:
input_text = encode_input(['Go.'],input_encoder)

In [19]:
sentence = predict(input_text, target_decoder)
print(sentence)

Tous !

